## Session 7 : Infinite sequences

Let us start with a recap of the basics about lazy lists (or streams): their datatype definition, construction, and elimination.

In [ ]:
type 'a seq = Nil | Cons of 'a * (unit -> 'a seq)

exception Empty

let head = function
  | Nil -> raise Empty
  | Cons (x, _) -> x

let tail = function
  | Nil -> raise Empty
  | Cons (_, xf) -> xf ()  

(* Note that here we need to call xf to obtain the tail sequence *)

Natural numbers and the Fibonacci sequence are easy to construct.

In [ ]:
(* Natural numbers from n *)
let rec from k = Cons(k, fun () -> from (k + 1))

(* Fibonacci sequence *)
let rec fib a b = Cons(a, fun () -> fib b (a + b))

We can also implement the [Collatz conjecture](https://en.wikipedia.org/wiki/Collatz_conjecture), which states that:
> Given a positive natural number n, do the following procedure:
>   - If n is even, divide it by 2.
>   - If n is odd, multiply it by 3 and then add 1.
>
> Conjecture: any sequence starting with n will lead to 1.

In [ ]:
let rec collatz n = 
  let xf () = 
    if n = 1 then Nil
    else if n mod 2 = 0 then collatz (n / 2)
    else collatz (3 * n + 1)
  in
    Cons(n, xf)

We use `get n seq` to extract the first `n` elements from a sequence.

In [ ]:
let rec get n seq = 
  match (n, seq) with
  | 0, _ -> []
  | n, Nil -> []
  | n, Cons(x, xf) -> x :: get (n - 1) (xf ())


We can modify `get` to verify Collatz conjecture for a given positive integer, up to a certain number of steps.
> `get_until n p seq` gets elements until one of the following conditions is met:
>  - `seq` is empty
>  - An element satisfying `p` is found
>  - Got a total of `n` elements

In [ ]:
let rec get_until n p seq = 
  match (n, seq) with 
  | 0, _ -> []
  | n, Nil -> []
  | n, Cons(x, xf) ->
    if p x then [x]
    else x :: get_until (n - 1) p (xf ())

In [ ]:
let try_collatz n = get_until 1000 (fun x -> x = 1) (collatz n)

We now review the following useful operations on sequences:
> `interleave : 'a seq -> 'a seq -> 'a seq`
>
> `filterq : ('a -> bool) -> 'a seq -> 'a seq`
>
> `mapq : ('a -> 'b) -> 'a seq -> 'b seq`
>
> `append : 'a seq -> 'a seq -> 'a seq`

In [ ]:
(* Interleaving two sequences *)
let rec interleave xq yq =
  match xq with
  | Nil -> yq
  | Cons(x, xf) -> Cons(x, fun () -> interleave yq (xf ()))

(* Filter *)
let rec filterq p = function
  | Nil -> Nil
  | Cons(x, xf) -> 
    if p x then Cons(x, fun () -> filterq p (xf ()))
    else filterq p (xf ())

(* Map *)
let rec mapq f = function
  | Nil -> Nil
  | Cons(x, xf) -> Cons(f x, fun () -> mapq f (xf ()))

(* Append *)
let rec appendq xq yq = 
  match xq with
  | Nil -> yq
  | Cons(x, xf) -> Cons(x, fun () -> appendq (xf ()) yq)

Now it's time for some exercise.
1. Code the lazy list whose elements are all ordinary lists of zeroes and ones, namely `[]; [0]; [1];
 [0; 0]; [0; 1]; [1; 0]; [1; 1]; [0; 0; 0];…`.

To do this, we first define a helper function `iter_seq f q` that returns `q @ (f q) @ (f (f q)) @ ...`. Then, we just need to find the suitable `f` to plug-in.

In [ ]:
let rec iter_seq f q = 
  match q with
  | Nil -> Nil
  | Cons(x, xf) -> Cons(x, fun () -> appendq (xf ()) (iter_seq f (f q)))

let bins = 
  let gen q = 
    appendq 
      (mapq (fun ls -> 0 :: ls) q)
      (mapq (fun ls -> 1 :: ls) q)
  in iter_seq gen (Cons([], fun () -> Nil))


2. A palindrome is a list that equals its own reverse. Code the lazy
 list whose elements are all palindromes of 0s and 1s, namely `[]; [0]; [1]; [0; 0]; [0; 0; 0];
 [0; 1; 0]; [1; 1]; [1; 0; 1]; [1; 1; 1]; [0; 0; 0; 0];, …`.

This is a simple use of `filterq`.

In [ ]:
(* List of palindromes *)
let palin = filterq (fun ls -> ls = List.rev ls) bins

3. Code a function to make change using lazy lists, delivering the sequence of all possible ways of
 making change. Using sequences allows us to compute solutions one at a time when there exists an
 astronomical number. Represent lists of coins using ordinary lists.

In [ ]:
(* Making changes *)
let rec change coins due =
  match coins with
  | [] -> Nil
  | c :: cs -> 
    if due = 0 then 
      Cons([], fun () -> Nil)
    else if c > due then
      change cs due
    else
      let q1 = change cs due in
      let q2 = mapq (fun ls -> c :: ls) (change coins (due - c)) in
      appendq q1 q2